In [ ]:
import pandas as pd
from pyprojroot import here
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data = pd.read_csv(here() / 'data/base/data.csv')

In [ ]:
categories = ['Very important', 'Important', 'Moderately important', 'Slightly important', 'Unimportant'][::-1]

In [ ]:
for c in data.iloc[:,30:60].columns:
    data[c] = pd.Categorical(data[c], categories)

In [ ]:
df = data.iloc[:,86:89]

In [ ]:
# Hospital level
a = list()

for c in df.columns:
    b = data.groupby(['Country', 'Hospital Name'])[c].value_counts(normalize=True)
    b.name = c
    a.append(b)

d = pd.concat(a, axis=1)

d = d[d.any(axis=1)]

d.index.names = ['Country', 'Hospital Name', 'Categories']

In [ ]:
# Country level
unique_hospitals = d.reset_index().groupby('Country')['Hospital Name'].nunique()
unique_hospitals.name = 'Country'

e = d.groupby(['Country', 'Categories']).sum()
a = list()

for c in e.columns:
    f = e[c].div(unique_hospitals, level='Country')
    f.name = c
    a.append(f)

g = pd.concat(a, axis=1)

In [ ]:
# EU level
from conversion import population
population = pd.Series(population)
weight = population / population.sum()

m = g.index.get_level_values(0).map(weight)
h = g.multiply(m, axis=0)
i = h.groupby('Categories').sum()

In [ ]:
table = i.loc[categories]

In [ ]:
a = (table / table.sum()).round(2) * 100
a = a.T

In [ ]:
a.index = ['Input', 'Throughput', 'Output']
a.columns = a.columns.str.replace(' ', '\n')

In [ ]:
fig, ax = plt.subplots(figsize=(4,2))

ax.imshow(a, cmap='Greys', vmin=0, vmax=100, aspect='auto')

ax.set_yticks(np.arange(len(a.index)))
ax.set_yticklabels(a.index)

ax.set_xticks(np.arange(len(a.columns)))
ax.set_xticklabels(a.columns)
plt.setp(ax.get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor')

arr = a.values 
for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        if arr[i,j]<80:
            color = 'black'
        else:
            color='white'
        ax.text(j, i, f"{arr[i, j]:.0f}", ha="center", va="center", color=color)

plt.savefig(here() / 'output/plots/enabled.png', dpi=300, bbox_inches='tight')